In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AI/train_clean.csv")

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
df.drop('Content', axis=1, inplace=True)

In [ ]:
df.head()

,Label,review_tokenized
0,1,second count input receiv intern control devic...
1,3,extrem low temperatur chip cold season chip li...
2,2,basic ammonium salt carboxyl eat water solubl ...
3,8,u provid axial support crankshaft via radial i...
4,1,uc inner surfac ud mean surfac layer away oute...


In [ ]:
df.columns = ['labels', 'data']

In [ ]:
df.head()

,labels,data
0,1,second count input receiv intern control devic...
1,3,extrem low temperatur chip cold season chip li...
2,2,basic ammonium salt carboxyl eat water solubl ...
3,8,u provid axial support crankshaft via radial i...
4,1,uc inner surfac ud mean surfac layer away oute...


In [ ]:
df["data"] = df["data"].astype(str)

In [ ]:
Y = df['labels'].values

In [ ]:
Y

array([1, 3, 2, ..., 6, 6, 6])

In [ ]:
# split up the data
df_train, df_test, Ytrain, Ytest = train_test_split(df['data'], Y, test_size=0.35)

In [ ]:
df_train

22336    level estim gener differ time frequenc direct ...
14800    exemplari embodi pipe form partit wall partit ...
8195     respons acceler pedal posit variou control fun...
58894    content ca mg ion respect mg l treat emuls min...
31029    experiment determin spectrum fragment model ta...
                               ...                        
14260    particular environ do mere purpos illustr unde...
5009     mean molecular weight liquefi solut determin f...
5291     within post lock tenon within slat slot pin ch...
20167    acet purifi flash silica column hexan ethyl ac...
4049     portion expand provid refriger liquid product ...
Name: data, Length: 38427, dtype: object

In [ ]:
# vectorizer = TfidfVectorizer(max_features=25000)  # Adjust max_features as needed
# X_train_tfidf = vectorizer.fit_transform(df_train)
# X_test_tfidf = vectorizer.transform(df_test)

In [ ]:
# T = X_train_tfidf.shape[1]

In [ ]:
# T

In [ ]:
# model = XGBClassifier(n_estimators=200, max_depth=10,learning_rate=0.1)  # Adjust n_estimators as needed

# # Train the model
# model.fit(X_train_tfidf, Ytrain)

# # Make predictions on test set
# y_pred = model.predict(X_test_tfidf)

# # Evaluate model performance
# accuracy = accuracy_score(Ytest, y_pred)

In [ ]:
MAX_VOCAB_SIZE = 9000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)

tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

In [ ]:
word2idx = tokenizer.word_index
V = len(word2idx)
print('Found %s unique tokens.' % V)

Found 50966 unique tokens.


In [ ]:
# vocabulary = vectorizer.get_feature_names_out()

In [ ]:
# V=len(vocabulary)

In [ ]:
# V
# print('Found %s unique tokens.' % V)

In [ ]:
# pad sequences so that we get a N x T matrix
data_train = pad_sequences(sequences_train)
print('Shape of data train tensor:', data_train.shape)

# get sequence length
T = data_train.shape[1]

Shape of data train tensor: (38427, 273)


In [ ]:
data_test = pad_sequences(sequences_test, maxlen=T)
print('Shape of data test tensor:', data_test.shape)

Shape of data test tensor: (20692, 273)


In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
# D = 30
# M=15
# i = Input(shape=(T,))
# x = Embedding(V + 1, D)(i)
# x = LSTM(M, return_sequences=True)(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.2)(x)
# x = GlobalMaxPooling1D()(x)
# x = Dense(9, activation='softmax')(x)

# model = Model(i, x)

In [ ]:
from keras import regularizers
l1_reg = regularizers.L1(l1=0.01)

In [ ]:
# X_train_tfidf

In [ ]:
# model = tf.keras.Sequential([
#   tf.keras.layers.Embedding(input_dim=vectorized_text.shape[1], output_dim=128, input_length=max_length),  # Adjust embedding dimensions
#   tf.keras.layers.LSTM(64, return_sequences=True),  # Adjust LSTM units
#   tf.keras.layers.LSTM(32),  # Adjust LSTM units
#   tf.keras.layers.Dense(len(your_classes), activation='softmax')  # Replace 'len(your_classes)' with actual number of classes
# ])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.layers import concatenate

In [ ]:
# Create the model

# We get to choose embedding dimensionality
D = 400
M=100
# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

def create_model():

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.
  ir = Input(shape=(T,))
  rnn_inputs=ir
  xr = Embedding(V + 1, D)(ir)
  xr = LSTM(M, return_sequences=True)(xr)
  xr = GlobalMaxPooling1D()(xr)
  rnn_outputs = xr




  i = Input(shape=(T,))
  cnn_inputs=i
  x = Embedding(V + 1, D)(i)
  x = Conv1D(128, 3, activation='relu')(x)
  x = GlobalMaxPooling1D()(x)
  cnn_outputs = x

  merged = concatenate([cnn_outputs, rnn_outputs])
  z = Dense(9, activation='softmax')(merged)
  ensemble_outputs = z



  model = Model(inputs=[cnn_inputs, rnn_inputs], outputs=ensemble_outputs)

  return model

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(
      optimizer='adam',
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])
# loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
batch_size = 128

# reshuffle_each_iteration=None is default but is later set to True if None
# thus "True" is the actual default

model.fit(
  [data_train, data_train], Ytrain,
  epochs=2,
  batch_size=batch_size,
  validation_data=([data_test, data_test], Ytest))

Epoch 1/2
301/301 [==============================] - 48s 125ms/step - loss: 1.5478 - accuracy: 0.4643 - val_loss: 1.2812 - val_accuracy: 0.5606
Epoch 2/2
301/301 [==============================] - 28s 94ms/step - loss: 1.0962 - accuracy: 0.6281 - val_loss: 1.2456 - val_accuracy: 0.5722


In [ ]:
D = 400
M=80
# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

def create_model2():

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.
  i = Input(shape=(T,))
  x = Embedding(V + 1, D)(i)
  x = Conv1D(128, 3, activation='relu')(x)
  x = GlobalMaxPooling1D()(x)
  x = Dense(9, activation='softmax')(x)

  model2 = Model(i, x)

  return model2

In [ ]:
with strategy.scope():
  model2 = create_model2()
  model2.compile(
      optimizer='adam',
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])
# loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
batch_size = 256

# reshuffle_each_iteration=None is default but is later set to True if None
# thus "True" is the actual default

model2.fit(
  data_train, Ytrain,
  epochs=4,
  batch_size=batch_size,
  validation_data=(data_test, Ytest))

Epoch 1/4
151/151 [==============================] - 18s 79ms/step - loss: 1.7269 - accuracy: 0.4053 - val_loss: 1.3992 - val_accuracy: 0.5214
Epoch 2/4
151/151 [==============================] - 4s 25ms/step - loss: 1.2241 - accuracy: 0.5857 - val_loss: 1.2762 - val_accuracy: 0.5632
Epoch 3/4
151/151 [==============================] - 4s 25ms/step - loss: 0.9954 - accuracy: 0.6613 - val_loss: 1.2676 - val_accuracy: 0.5674
Epoch 4/4
151/151 [==============================] - 4s 25ms/step - loss: 0.7846 - accuracy: 0.7428 - val_loss: 1.2965 - val_accuracy: 0.5664


In [ ]:
# model.compile(
#       optimizer='adam',
#       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#       metrics=['accuracy'])

In [ ]:
# train_dataset = data_train
# test_dataset = data_test

# model.fit(
#   data_train,
#   Ytrain,
#   epochs=20,
#   validation_data=(data_test, Ytest))

In [ ]:
# # Compile and fit
# model.compile(
#   loss='sparse_categorical_crossentropy',
#   optimizer='adam',
#   metrics=['accuracy']
# )


# print('Training model...')
# r = model.fit(
#   data_train,
#   Ytrain,
#   epochs=20,
#   validation_data=(data_test, Ytest)
# )

In [ ]:
df_val = pd.read_csv("/content/drive/MyDrive/AI/test_clean.csv")

In [ ]:
df_val.drop('Content', axis=1, inplace=True)

In [ ]:
X_val=df_val["review_tokenized"].astype(str)

In [ ]:
model.save('mymodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
sequences_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
data_val = pad_sequences(sequences_val, maxlen=T)

In [ ]:
with strategy.scope():
  model = tf.keras.models.load_model('mymodel.h5')
  out = model.predict(data_val)
  print(out)

792/792 [==============================] - 8s 9ms/step
[[4.3095555e-02 8.7772705e-02 5.4604546e-03 ... 7.8377230e-03
  4.2589465e-03 5.0042439e-02]
 [1.4199373e-01 1.4360642e-01 9.4205892e-04 ... 1.5140272e-02
  1.1331962e-02 8.9847706e-02]
 [9.2279625e-01 2.5179230e-02 3.0875737e-03 ... 1.7053655e-03
  2.9240965e-04 2.3364497e-02]
 ...
 [1.7997336e-01 8.9374319e-02 3.6678514e-03 ... 4.7179717e-03
  4.0374710e-03 7.5061359e-02]
 [1.3820392e-03 1.0209848e-01 3.8857651e-03 ... 1.3126433e-01
  6.0365176e-01 1.3159573e-01]
 [8.8031767e-03 1.2326389e-02 1.9256070e-03 ... 3.3007763e-02
  7.8024954e-01 4.8300188e-02]]


In [ ]:
predicted_labels = tf.argmax(out, axis=1)

In [ ]:
predicted_labels

<tf.Tensor: shape=(25337,), dtype=int64, numpy=array([5, 5, 0, ..., 3, 7, 7])>

In [ ]:
df_submission=pd.read_csv("/content/submission.csv")

In [ ]:
df_submission

,Label
0,6
1,2
2,1
3,1
4,3
...,...
25332,6
25333,8
25334,4
25335,8


In [ ]:
predicted_labels=predicted_labels+1

In [ ]:
df_submission["Label"]=predicted_labels
df_submission.to_csv('submission6.csv', index=False)